<a href="https://colab.research.google.com/github/denis911/pytorch-scikit-learn-uv-docker-serverless-model-deployment-aws-lambda/blob/main/HW9_ONNX_format_for_neural_network_models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Q1 - download the model using ONNX format

In [1]:
PREFIX="https://github.com/alexeygrigorev/large-datasets/releases/download/hairstyle"
DATA_URL=f"{PREFIX}/hair_classifier_v1.onnx.data"
MODEL_URL=f"{PREFIX}/hair_classifier_v1.onnx"
!wget {DATA_URL};
!wget {MODEL_URL};

--2025-12-07 20:13:36--  https://github.com/alexeygrigorev/large-datasets/releases/download/hairstyle/hair_classifier_v1.onnx.data
Resolving github.com (github.com)... 140.82.121.4
Connecting to github.com (github.com)|140.82.121.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://release-assets.githubusercontent.com/github-production-release-asset/426348925/398ded4a-c41c-4e5a-9672-acb7e441de54?sp=r&sv=2018-11-09&sr=b&spr=https&se=2025-12-07T20%3A48%3A07Z&rscd=attachment%3B+filename%3Dhair_classifier_v1.onnx.data&rsct=application%2Foctet-stream&skoid=96c2d410-5711-43a1-aedd-ab1947aa7ab0&sktid=398a6654-997b-47e9-b12b-9515b896b4de&skt=2025-12-07T19%3A47%3A59Z&ske=2025-12-07T20%3A48%3A07Z&sks=b&skv=2018-11-09&sig=3WJektxqQB%2FW2zon9qWsgPZJ7pIq6%2BpUElsazhgwoZw%3D&jwt=eyJ0eXAiOiJKV1QiLCJhbGciOiJIUzI1NiJ9.eyJpc3MiOiJnaXRodWIuY29tIiwiYXVkIjoicmVsZWFzZS1hc3NldHMuZ2l0aHVidXNlcmNvbnRlbnQuY29tIiwia2V5Ijoia2V5MSIsImV4cCI6MTc2NTE0MDIxNiwibmJmIjoxNzY1MTM4NDE2LCJ

In [7]:
# !pip install pillow # Requirement already satisfied

# to prepare images we need processing functions:

from io import BytesIO
from urllib import request

from PIL import Image

def download_image(url):
    with request.urlopen(url) as resp:
        buffer = resp.read()
    stream = BytesIO(buffer)
    img = Image.open(stream)
    return img


def prepare_image(img, target_size):
    if img.mode != 'RGB':
        img = img.convert('RGB')
    img = img.resize(target_size, Image.NEAREST)
    return img

In [8]:
# Model is saved in the ONNX format. Like with TF-lite, we only need ONNX-Runtime to run it:

!pip install onnxruntime --q # --q to prohibit listing

# This is how to use ONNX-Runtime to make predictions:

import onnxruntime as ort

onnx_model_path = "/content/hair_classifier_v1.onnx"
session = ort.InferenceSession(onnx_model_path, providers=["CPUExecutionProvider"])

inputs = session.get_inputs()
outputs = session.get_outputs()

input_name = inputs[0].name
output_name = outputs[0].name
output_name

# Question 1
# To be able to use this model, we need to know the name of the input and output nodes.
# What's the name of the output:

# output === output
# sigmoid
# softmax
# prediction

'output'

## Q2 - Target size

In [4]:
!pip install keras_image_helper --q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.1/62.1 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.6/16.6 MB 108.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
opencv-python-headless 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 2.3.5 which is incompatible.
numba 0.60.0 requires numpy<2.1,>=1.22, but you have numpy 2.3.5 which is incompatible.
opencv-python 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 2.3.5 which is incompatible.
opencv-contrib-python 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 2.3.5 which is incompatible.
tensorflow 2.19.0 requires numpy<2.2.0,>=1.26.0, but you have numpy 2.3.5 which is incompatible.


In [10]:
# Let's download and resize this image:

# https://habrastorage.org/webt/yf/_d/ok/yf_dokzqy3vcritme8ggnzqlvwa.jpeg

# Based on the previous homework, what should be the target size for the image?
# The shape for input should be (3, 200, 200) (channels first format in PyTorch)
# 64x64
# 128x128
# 200x200 === (3, 200, 200)
# 256x256

# Get the image:

import numpy as np
from keras_image_helper import create_preprocessor

def preprocess_pytorch(X):
    # X: shape (1, 299, 299, 3), dtype=float32, values in [0, 255]
    X = X / 255.0

    mean = np.array([0.485, 0.456, 0.406]).reshape(1, 3, 1, 1)
    std = np.array([0.229, 0.224, 0.225]).reshape(1, 3, 1, 1)

    # Convert NHWC → NCHW
    # from (batch, height, width, channels) → (batch, channels, height, width)
    X = X.transpose(0, 3, 1, 2)

    # Normalize
    X = (X - mean) / std

    return X.astype(np.float32)


preprocessor = create_preprocessor(preprocess_pytorch, target_size=(200, 200))

url = 'https://habrastorage.org/webt/yf/_d/ok/yf_dokzqy3vcritme8ggnzqlvwa.jpeg'
X = preprocessor.from_url(url)

In [11]:
# Make predictions:

result = session.run([output_name], {input_name: X})
result # [array([[0.09156627]], dtype=float32)] - probability of straight hair is 9%

[array([[0.09156627]], dtype=float32)]

Q3 - image pre-processing

In [13]:
# Question 3
# Now we need to turn the image into numpy array and pre-process it.
# After the pre-processing, what's the value in the first pixel, the R channel?

# -10.73
# -1.073 === -1.0732939
# 1.073
# 10.73

X[0][0]

array([[-1.0732939 , -1.0047948 , -1.0390444 , ..., -1.0732939 ,
        -1.0732939 , -1.210292  ],
       [-1.0561692 , -1.0219196 , -1.0047948 , ..., -1.0219196 ,
        -1.0561692 , -1.1760424 ],
       [-0.9534206 , -0.97054535, -0.9191711 , ..., -1.0219196 ,
        -1.1075435 , -1.2274168 ],
       ...,
       [-1.6726604 , -1.6726604 , -1.5185375 , ...,  1.7351657 ,
         1.649542  ,  1.7865399 ],
       [-1.6726604 , -1.6555356 , -1.6384108 , ...,  1.6837914 ,
         1.5981677 ,  1.7009162 ],
       [-1.6384108 , -1.7240345 , -1.6726604 , ...,  1.718041  ,
         1.7351657 ,  1.6837914 ]], dtype=float32)

In [15]:
# Question 4
# Now let's apply this model to this image. What's the output of the model?

# 0.09 === [[0.09156627]]
# 0.49
# 0.69
# 0.89
result

[array([[0.09156627]], dtype=float32)]